In [6]:
from QASMBench.interface.qiskit import QASMBenchmark

# path to the root directory of QASMBench
path = "QASMBench"

# selected category for QASMBench
category = "large" 

# select only the circuits with the number of qubits in the list
num_qubits_list = list(range(3, 50))

# whether to remove the final measurement in the circuit
remove_final_measurements = True

# whether to use qiskit.transpile() to transpile the circuits (note: must provide qiskit backend)
do_transpile = True  # Changed to True for transpiling circuits

# arguments for qiskit.transpile(). backend should be provided at least
transpile_args = { 'basis_gates': ['u', 'cp']}

bm = QASMBenchmark(path, category, num_qubits_list=num_qubits_list, do_transpile=do_transpile, **transpile_args, remove_final_measurements=remove_final_measurements)

In [7]:
circ_list = bm.circ_list

In [8]:
circ_name_list = bm.circ_name_list

In [9]:
for i in range(len(circ_list)):
    print(circ_name_list[i])
    print(circ_list[i].num_qubits)

adder_n28
28
qft_n29
29
bv_n30
30
knn_n31
31
cc_n32
32
QV_n32
32
dnn_n33
33
ising_n34
34
cat_n35
35
wstate_n36
36
bwt_n37
37
qugan_n39
39
ghz_n40
40
swap_test_n41
41
knn_n41
41
ising_n42
42
multiplier_n45
45
square_root_n45
45


In [ ]:
import os
import json
import numpy as np
from qiskit import transpile
from MLFM_GCP.partitioning.fgp.fgp_roee import *
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_fgp_QASM_5it.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_results_fgp_QASM_5it_means.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

num_partition_list = [2,3,4]

buffer = 0
for num_partitions in num_partition_list:
    for i, circ in enumerate(circ_list[buffer:]):
        # For each increase of 8 qubits, increase the number of partitions by 1
        i = i + buffer
        num_qubits = circ.num_qubits
        circuit = circ
        circuit = transpile(circuit, basis_gates = ['u', 'cp'])
        qpu_sizes = [int(num_qubits/num_partitions) + 1 for _ in range(num_partitions)]

        if circuit.depth() < 1000:
            print(f"Starting benchmark for {num_qubits} qubits, {num_partitions} partitions, circuit: {circ_name_list[i]}")

            initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
            # Create an All-to-All network
            iteration_data = []
            for iteration in range(5):
                
                # -------------------------
                # 1. Define/redefine circuit
                # -------------------------

                # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
                # circuit = QAOA_random(num_qubits,prob=0.5, reps=1)

                
                # -------------------------
                # 5. Recursive refinement
                # -------------------------
                try:
                    start = time.time()
                    partition, cost, mapping = main_algorithm(circuit=circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=False)
                    stop = time.time()

                    total_time_fgp = stop - start
                    min_cost_fgp = cost
                except:
                    ValueError
                    total_time_fgp = 0
                    min_cost_fgp = 0
                    print("Error in FGP")

                total_time_fgp = stop - start
                min_cost_fgp = cost

                
                # -------------------------
                # 6. Store iteration-level results
                # -------------------------
                result_entry = {
                    "circuit_name": circ_name_list[i],
                    "num_qubits": num_qubits,
                    "num_partitions": num_partitions,
                    "iteration": iteration,
                    "fgp_cost":  min_cost_fgp,
                    "time_fgp": total_time_fgp,
                }
                
                detailed_results.append(result_entry)
                iteration_data.append(result_entry)
                
                # Update detailed JSON right away
                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)
            
            # ---------------------------------------------------------------------
            # After 10 iterations, compute the means and log them
            # ---------------------------------------------------------------------

            fgp_cost_list = [x["fgp_cost"] for x in iteration_data]
            

            fgp_time_list = [x["time_fgp"] for x in iteration_data]
            

            mean_fgp_cost = float(np.mean(fgp_cost_list))

            mean_fgp_time = float(np.mean(fgp_time_list))
            
            # Print to console for quick logging
            print("=============================================")
            print(f"Finished 10 iterations for:")
            print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, circuit: {circ_name_list[i]}")
            print("Mean Costs:")

            print(f"  FGP:{mean_fgp_cost:.3f}")
            print("Mean Times (s):")

            print(f"  FGP:{mean_fgp_time:.3f}")
            print("=============================================")
            
            # Store the aggregated means in a separate JSON
            mean_entry = {
                "circuit_name": circ_name_list[i],
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "mean_fgp_cost": mean_fgp_cost,
                "mean_fgp_time": mean_fgp_time,
            }
            
            mean_results.append(mean_entry)
            
            # Update the means JSON file
            with open(means_filename, "w") as f:
                json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Starting benchmark for 28 qubits, 2 partitions, circuit: adder_n28
Finished 10 iterations for:
  # Qubits: 28, # Partitions: 2, circuit: adder_n28
Mean Costs:
  FGP:8.000
Mean Times (s):
  FGP:0.029
Starting benchmark for 29 qubits, 2 partitions, circuit: qft_n29
Finished 10 iterations for:
  # Qubits: 29, # Partitions: 2, circuit: qft_n29
Mean Costs:
  FGP:46.000
Mean Times (s):
  FGP:0.123
Starting benchmark for 30 qubits, 2 partitions, circuit: bv_n30
Error in FGP
Error in FGP
Error in FGP
Error in FGP
Error in FGP
Finished 10 iterations for:
  # Qubits: 30, # Partitions: 2, circuit: bv_n30
Mean Costs:
  FGP:46.000
Mean Times (s):
  FGP:-0.024
Starting benchmark for 31 qubits, 2 partitions, circuit: knn_n31
Finished 10 iterations for:
  # Qubits: 31, # Partitions: 2, circuit: knn_n31
Mean Costs:
  FGP:26.000
Mean Times (s):
  FGP:0.021
Starting benchmark for 32 qubits, 2 partitions, circuit: cc_n32
Error in FGP
Error in FGP
Error in FGP
Error in FGP
Error in FGP
Finished 10 iteratio

{'circuit_name': 'adder_n28', 'num_qubits': 28, 'num_partitions': 2, 'mean_r_cost': 2.6, 'mean_r_time': 5.32185115814209, 'ebit_frac_part': 0.013333333333333334}
{'circuit_name': 'qft_n29', 'num_qubits': 29, 'num_partitions': 2, 'mean_r_cost': 14.0, 'mean_r_time': 6.3917553424835205, 'ebit_frac_part': 0.017241379310344827}
{'circuit_name': 'bv_n30', 'num_qubits': 30, 'num_partitions': 2, 'mean_r_cost': 1.8, 'mean_r_time': 0.7240607261657714, 'ebit_frac_part': 0.1}
{'circuit_name': 'knn_n31', 'num_qubits': 31, 'num_partitions': 2, 'mean_r_cost': 1.0, 'mean_r_time': 3.2066197872161863, 'ebit_frac_part': 0.008333333333333333}
{'circuit_name': 'cc_n32', 'num_qubits': 32, 'num_partitions': 2, 'mean_r_cost': 2.6, 'mean_r_time': 3.053362989425659, 'ebit_frac_part': 0.08125}
{'circuit_name': 'QV_n32', 'num_qubits': 32, 'num_partitions': 2, 'mean_r_cost': 218.6, 'mean_r_time': 5.274758815765381, 'ebit_frac_part': 0.14231770833333332}
{'circuit_name': 'dnn_n33', 'num_qubits': 33, 'num_partitions

Total sum of ebit fractions: 0.0704701919990416
